In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn import metrics
import time

In [2]:
%%time
mnist = fetch_openml("mnist_784")

CPU times: user 14 s, sys: 293 ms, total: 14.3 s
Wall time: 14.6 s


In [3]:
mnist.data.shape

(70000, 784)

In [4]:
mnist.target.shape

(70000,)

In [5]:
print(type(mnist.data), type(mnist.target))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [6]:
images = mnist.data
targets = mnist.target

In [7]:
data_df = pd.DataFrame(images)
label_df = pd.DataFrame(targets)

In [8]:
# data_df

In [9]:
# label_df

In [10]:
# normalization
# scaler = StandardScaler()
scaler = RobustScaler()
scaler.fit(images)
X_scaled = scaler.transform(images)

In [11]:
# stratify sampling
X_train, X_test, y_train, y_test = train_test_split(X_scaled, targets, test_size=0.3, random_state=123, stratify=targets)

In [12]:
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(49000, 784) (21000, 784)
(49000,) (21000,)


In [13]:
X_train_df = pd.DataFrame(X_train)
y_train_df = pd.DataFrame(y_train)

In [14]:
# X_train_df

In [15]:
# y_train_df

# SVM - linear kernel

In [16]:
from sklearn.svm import SVC

In [17]:
C = [1000.0, 100.0, 10.0, 1.0, 0.1, 0.01, 0.001]

In [19]:
def run(X_train, X_test, y_train, y_test, C):
    parameters = {'kernel':['linear'], 'C':C}
    
    # training
#     svc = SVC(kernel="linear", C=C)
    svc = SVC()
    clf = GridSearchCV(estimator=svc, param_grid=parameters, scoring='accuracy', cv=10, n_jobs=4)
    %time clf.fit(X_train, y_train)

    print(clf.cv_results_["mean_test_score"])
    print(clf.best_params_, " : \n", clf.best_score_)
    
#     # classify
#     predict_start = time.time()
#     y_pred_svc = clf[i].predict(X_test)
#     predict_end = time.time()
#     print("predict time : %2f" %(predict_end - predict_start))
    
#     acc = metrics.accuracy_score(y_test, y_pred_svc)
#     print("Accuracy: %2f" %acc)
#     print("\n")

    return clf.cv_results_["mean_test_score"]

In [20]:
def plot(x, y):
    plt.title("MNIST classify")
    plt.bar(x, y)
    xlabel = ['1000.0', '100.0', '10.0', '1.0', '0.1', '0.01', '0.001']
    plt.xticks(x, xlabel)
    plt.yticks(y)
    plt.xlabel("C")
    plt.ylabel("Accuracy")
    plt.show()

In [21]:
%%time
scores = run(X_train, X_test, y_train, y_test, sorted(C, reverse=True))
# plot(C, socres)

KeyboardInterrupt: 

AttributeError: 'GridSearchCV' object has no attribute 'cv_results_'

In [ ]:
# number of supports in each class
# clf.n_support_

In [ ]:
# index of supports in each class
# clf.support_

In [ ]:
# x value of suppport in each class
# clf.support_vectors_

In [ ]:
# w vector
# clf.coef_

In [ ]:
# -w0
# clf.intercept_